In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
from nltk import tokenize
import random
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
df=  pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/train.tsv.zip', sep = '\t')

In [ ]:
df.head()

In [ ]:
def tokenizing_and_label_encoding(inputs , labels):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(inputs)
    seq = tokenizer.texts_to_sequences(inputs)
    
    print('len vocab :', len(tokenizer.word_index))
    
    max_sentences_len = np.max([*map(lambda x: len(x) , seq)])
    
    print('max_len :', max_sentences_len)
    
    sequences = pad_sequences(seq, maxlen=max_sentences_len, padding='post')
    
    #----------------------
    labels = np.array(labels)
    
    return sequences, labels, tokenizer, max_sentences_len

def preprocessing_data(inputs , labels):
    X_train, X_test, y_train, y_test = train_test_split(inputs, labels, 
                                                                        train_size=0.75, 
                                                                        shuffle=True, 
                                                                        random_state=1) 
    return X_train, X_test, y_train , y_test

In [ ]:
X, y , tokenizer_ , max_len = tokenizing_and_label_encoding(df.Phrase.tolist() , df.Sentiment.tolist() )

In [ ]:
print(X[0])
print(df.Phrase.loc[0])

In [ ]:
X_train, X_test, y_train, y_test = preprocessing_data(X,y)

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf


inputs = tf.keras.Input(shape=(X_train.shape[1],))

x = tf.keras.layers.Embedding(input_dim=15289, #length of word index dictionary + 1 (void)
                              output_dim=20,
                              input_length=X_train.shape[1])(inputs)

x = tf.keras.layers.GRU(32, return_sequences=True, activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=30,
    
)


# test data

In [ ]:
df_test =  pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/test.tsv.zip', sep = '\t')

In [ ]:
df_samp = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/sampleSubmission.csv')

In [ ]:
test_X = df_test.Phrase.tolist()
test_y = df_samp.Sentiment.tolist()

In [ ]:
test_X = tokenizer_.texts_to_sequences(test_X)
test_X = np.array(pad_sequences(test_X, maxlen=max_len, padding='post'))

test_y = np.array(test_y)

In [ ]:
y_pred = model.predict(test_X)
y_pred = np.argmax(y_pred , axis = 1)


In [ ]:
acc = np.sum(test_y == y_pred)/y_pred.shape[0]

In [ ]:
print('accuracy :', acc)

In [ ]:
df_samp['Prediction'] = y_pred

In [ ]:
df_samp.to_csv('submission.csv')